# Лабораторна робота 2

## Порівняння методів класифікації даних

### Завдання 2.2. Порівняння якості класифікаторів SVM з нелінійними ядрами

Завантажуємо необхідні бібліотеки, класи та функції

In [7]:
import pandas as pd
import numpy as np
import time
from tabulate import tabulate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import accuracy_score, classification_report

Читаємо дані з файлу та очищаємо їх

In [8]:
# Читаємо дані з CSV файлу
df = pd.read_csv('income_data.csv', header=None, skipinitialspace=True)
df.describe()

,0,2,4,10,11,12
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [9]:
count_class1 = len(df[df[:-1] == "<=50K"])
count_class2 = len(df[df[:-1] == ">50K"])
print(f"Кількість рядків з класом <=50K: {count_class1}")
print(f"Кількість рядків з класом >50K: {count_class2}")

Кількість рядків з класом <=50K: 32561
Кількість рядків з класом >50K: 32561


Готуємо дані для аналізу

In [10]:
# Кодуємо категоріальні ознаки (стовпці)
df_encoded = df.copy()
label_encoders = {}

for col in df_encoded.columns:
    if df_encoded[col].dtype == 'object' or df_encoded[col].dtype.name == 'category':
        le = LabelEncoder()
        df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
        label_encoders[col] = le

# Розділяємо дані на ознаки та мітки
X = df_encoded.iloc[:, :-1]
y = df_encoded.iloc[:, -1] # Останній стовпець - це мітка класу

Розбиваємо набір дани на тренувальний і перевірочний набори

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Тренуємо різні моделі SVM, обчислюємо їх точність, та заміряємо час

In [ ]:
models = {
    'LinearSVC': LinearSVC(max_iter=10000),
    'SVC (poly kernel)': SVC(kernel='poly'),
    'SVC (rbf kernel)': SVC(kernel='rbf'), # default kernel (Gaussian RBF)
    'SVC (sigmoid kernel)': SVC(kernel='sigmoid')
}


In [13]:
# Навчаємо кожну модель та вимірюємо час тренування
model_results = {}
for model_name, model in models.items():
    # Train the model and measure time
    start_train = time.time()
    model.fit(X_train, y_train)
    end_train = time.time()

    model_results[model_name] = {"train_time": end_train - start_train}


In [14]:
# Тестуємо кожну модель та оцінюємо їх продуктивність
for model_name, model in models.items():
    # Predict and measure time
    start_pred = time.time()
    y_pred = model.predict(X_test)
    end_pred = time.time()

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    rpt = classification_report(y_test, y_pred, output_dict=True)

    model_results[model_name]["accuracy"] = accuracy
    model_results[model_name]["predict_time"] = end_pred - start_pred
    model_results[model_name]["macro avg precision"] = rpt["macro avg"]["precision"]
    model_results[model_name]["macro avg recall"] = rpt["macro avg"]["recall"]
    model_results[model_name]["macro avg f1-score"] = rpt["macro avg"]["f1-score"]


Виводимо результати

In [15]:
# Форматуємо результати для виводу
model_results_output = []
for model_name in model_results.keys():
    result = model_results[model_name]
    model_results_output.append(
        [
            model_name,
            result["accuracy"],
            result["macro avg precision"],
            result["macro avg recall"],
            result["macro avg f1-score"],
            result["train_time"],
            result["predict_time"],
        ]
    )

# Виводимо результати в табличному форматі
print(
    tabulate(
        model_results_output,
        headers=[
            "Model",
            "Accuracy",
            "Macro Avg Precision",
            "Macro Avg Recall",
            "Macro Avg F1-score",
            "Train Time (sec)",
            "Predict Time (sec)",
        ],
        tablefmt="orgtbl",
    )
)


| Model                |   Accuracy |   Macro Avg Precision |   Macro Avg Recall |   Macro Avg F1-score |   Train Time (sec) |   Predict Time (sec) |
|----------------------+------------+-----------------------+--------------------+----------------------+--------------------+----------------------|
| LinearSVC            |   0.806848 |              0.799344 |           0.62415  |             0.644274 |          0.0528553 |            0.0118611 |
| SVC (rbf kernel)     |   0.795793 |              0.886125 |           0.577571 |             0.575272 |          4.41242   |            2.40702   |
| SVC (sigmoid kernel) |   0.653462 |              0.520797 |           0.520255 |             0.520463 |          4.87242   |            0.890131  |
